In [37]:
import pandas as pd
from collections import Counter
import re
import matplotlib.pyplot as plt
import numpy as np

plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

comp_color = {
    '魔坦刺': '#858561',
    '扶桑法刺': '#9e71ad',
    '扶桑群雄刺': '#968795',
    '扶桑群雄法刺': '#6603fc',
    '扶桑刺': '#6603fc',
    '魔种': '#a35956',
    '扶桑男刺': '#bfae91',
    '扶桑法': '#8000ff',

    '七射': '#14ffa5',
    '攻辅射': '#42d7f5',
    '弟弟射': '#fceb30',
    '攻辅蛋': '#ffac1c',

    '九战': '#ff0000',
    '稷下战': '#ff5e5e',
    '轻战守约': '#ff8e5e',
    '魏战': '#ff0084',
    '长城战': '#f8ff6b',

    '蜀国': '#00cc18',
    '弟弟蜀': '#a1eb34',

    '法奶': '#0093f5',
    '弟妹法': '#7900fa',
    '坦法': '#ff5900'
    }

comp_to_brief = {
    '扶桑法刺': '扶桑刺',
    '扶桑刺': '扶桑刺',
    '扶桑群雄刺': '扶桑刺',
    '扶桑群雄法刺': '扶桑刺',
    '扶桑男刺': '扶桑刺',
    '魔种(无天赋)' :'魔种',
    '魔种(有天赋)' :'魔种',
    '封神攻辅射': '攻辅射',
    '吴国攻辅射': '攻辅射',
    '稷下攻辅射': '攻辅射',
    '尧天攻辅射': '攻辅射'
}

rank_to_score = [12, 9, 7, 5, 3, 2, 1, 0]

class Config:
    use_brief_comp = True

pd.set_option('display.max_columns', None)

In [38]:
def preprocess_data(df):
    df = df.dropna()
    df['分数'] = df['排名'].apply(lambda x: rank_to_score[int(x) - 1])
    if Config.use_brief_comp:
        df['详细阵容'] = df['阵容']
        df['阵容'] = df['详细阵容'].apply(lambda x: comp_to_brief.get(x, x))
    df['回合数'] = df['胜'] + df['负']
    return df

def check_data_integrity(df):
    groups = df.groupby(['周数', '分组', '局数'])
    for g_name, g_df in groups:
        assert g_df['排名'].tolist() == [1, 2, 3, 4, 5, 6, 7, 8], g_df['排名'].tolist()
        assert g_df['回合数'].is_monotonic_decreasing, g_df
        tmp_l = g_df['回合数'].tolist()
        assert tmp_l[0] == tmp_l[1], g_df['回合数']
        assert len(g_df['俱乐部'].unique()) == len(g_df['俱乐部']), len(g_df['俱乐部'].unique())

    # 全局BP
    groups = df.groupby(['周数', '分组', '俱乐部'])
    for g_name, g_df in groups:
        assert len(g_df['ban位'].unique()) == len(g_df['ban位']), (g_name, len(g_df['ban位'].unique()))

    return df

df = pd.read_excel('result.xlsx')
df = preprocess_data(df)
df = check_data_integrity(df)

In [39]:
def get_comp_statistics(df):
    def get_most_ban(x):
        c = Counter(x['ban位'])
        return [(i, c[i] / len(x)) for i, count in c.most_common()]

    groups = df.groupby('阵容')
    mean_rank = groups['排名'].mean()
    var_rank = groups['排名'].var()
    sum_occur = groups['排名'].count()
    mean_score = groups['分数'].mean()
    top1_cnt = groups['排名'].apply(lambda x: (x == 1).sum())
    top1_rate = top1_cnt / sum_occur
    top3_cnt = groups['排名'].apply(lambda x: (x <= 3).sum())
    top3_rate = top3_cnt / sum_occur
    most_ban = groups.apply(get_most_ban)
    res = pd.concat([mean_score, mean_rank, var_rank, sum_occur, top1_cnt, top1_rate, top3_cnt, top3_rate, most_ban], axis=1)
    res.columns = ('平均分数', '平均排名', '排名方差', '出场次数', '登顶次数', '登顶率', '前三次数', '前三率', '阵容选用')
    res = res.sort_values(['平均分数', '平均排名', '登顶率', '前三率'], ascending=[False, True, False, False])
    res = res.fillna('-')
    return res

def get_ban_statistics(df):
    def get_most_comp(x):
        c = Counter(x['阵容'])
        return [(i, c[i]) for i, count in c.most_common()]
    groups = df.groupby('ban位')
    mean_rank = groups['排名'].mean()
    var_rank = groups['排名'].var()
    mean_score = groups['分数'].mean()
    sum_occur = groups['排名'].count()
    top1_cnt = groups['排名'].apply(lambda x: (x == 1).sum())
    top1_rate = top1_cnt / sum_occur
    top3_cnt = groups['排名'].apply(lambda x: (x <= 3).sum())
    top3_rate = top3_cnt / sum_occur
    most_comp = groups.apply(get_most_comp)
    res = pd.concat([mean_score, mean_rank, var_rank, sum_occur, top1_cnt, top1_rate, top3_cnt, top3_rate, most_comp], axis=1)
    res.columns = ('平均分数', '平均排名', '排名方差', '出场次数', '登顶次数', '登顶率', '前三次数', '前三率', '阵容选用')
    res = res.sort_values(['平均分数', '平均排名', '登顶率', '前三率'], ascending=[False, True, False, False])
    return res


def get_team_statistics(df):
    def get_most_comp(x):
        c = Counter(x['阵容'])
        return [(i, c[i]) for i, count in c.most_common()]
    groups = df.groupby('俱乐部')
    mean_rank = groups['排名'].mean()
    var_rank = groups['排名'].var()
    mean_score = groups['分数'].mean()
    sum_occur = groups['排名'].count()
    top1_cnt = groups['排名'].apply(lambda x: (x == 1).sum())
    top1_rate = top1_cnt / sum_occur
    top3_cnt = groups['排名'].apply(lambda x: (x <= 3).sum())
    top3_rate = top3_cnt / sum_occur
    most_comp = groups.apply(get_most_comp)
    res = pd.concat([mean_score, mean_rank, var_rank, sum_occur, top1_cnt, top1_rate, top3_cnt, top3_rate, most_comp], axis=1)
    res.columns = ('平均分数', '平均排名', '排名方差', '出场次数', '登顶次数', '登顶率', '前三次数', '前三率', '阵容选用')
    res = res.sort_values(['平均分数', '平均排名', '登顶率', '前三率'], ascending=[False, True, False, False])
    return res

def get_player_statistics(df):
    def get_most_comp(x):
        c = Counter(x['阵容'])
        return [(i, c[i]) for i, count in c.most_common()]
    groups = df.groupby('选手')
    mean_rank = groups['排名'].mean()
    var_rank = groups['排名'].var()
    mean_score = groups['分数'].mean()
    sum_occur = groups['排名'].count()
    top1_cnt = groups['排名'].apply(lambda x: (x == 1).sum())
    top1_rate = top1_cnt / sum_occur
    top3_cnt = groups['排名'].apply(lambda x: (x <= 3).sum())
    top3_rate = top3_cnt / sum_occur
    most_comp = groups.apply(get_most_comp)
    res = pd.concat([mean_score, mean_rank, var_rank, sum_occur, top1_cnt, top1_rate, top3_cnt, top3_rate, most_comp], axis=1)
    res.columns = ('平均分数', '平均排名', '排名方差', '出场次数', '登顶次数', '登顶率', '前三次数', '前三率', '阵容选用')
    res = res.sort_values(['平均分数', '平均排名', '登顶率', '前三率'], ascending=[False, True, False, False])
    return res

display(get_comp_statistics(df))
display(get_ban_statistics(df))
display(get_team_statistics(df))
display(get_player_statistics(df))

,平均分数,平均排名,排名方差,出场次数,登顶次数,登顶率,前三次数,前三率,阵容选用
阵容,,,,,,,,,
攻辅蛋,9.000000,2.000000,-,1,0,0.000000,1,1.000000,"[(蜀, 1.0)]"
弟弟射,6.777778,3.444444,5.2254,36,9,0.250000,21,0.583333,"[(蜀, 1.0)]"
攻辅射,5.600000,4.133333,6.11954,30,5,0.166667,15,0.500000,"[(蜀, 0.3333333333333333), (卫, 0.33333333333333..."
扶桑法,5.000000,4.000000,-,1,0,0.000000,0,0.000000,"[(卫, 1.0)]"
长城战,5.000000,4.000000,-,1,0,0.000000,0,0.000000,"[(安, 1.0)]"
魏战,5.000000,4.000000,-,1,0,0.000000,0,0.000000,"[(安, 1.0)]"
扶桑刺,4.969072,4.443299,5.37436,97,12,0.123711,36,0.371134,"[(卫, 0.36082474226804123), (稷, 0.3505154639175..."
稷下战,4.728814,4.508475,4.39217,59,6,0.101695,20,0.338983,"[(安, 0.6440677966101694), (魏, 0.16949152542372..."
轻战守约,4.666667,5.000000,13,3,1,0.333333,1,0.333333,"[(安, 0.6666666666666666), (吴, 0.33333333333333..."


,平均分数,平均排名,排名方差,出场次数,登顶次数,登顶率,前三次数,前三率,阵容选用
ban位,,,,,,,,,
蜀,6.203125,3.750000,5.269841,64,13,0.203125,34,0.531250,"[(弟弟射, 36), (攻辅射, 10), (稷下战, 6), (法奶, 5), (扶桑刺..."
安,4.925926,4.462963,5.272187,54,8,0.148148,20,0.370370,"[(稷下战, 38), (蜀国, 7), (九战, 3), (轻战守约, 2), (魔种, ..."
卫,4.890625,4.421875,4.628720,64,6,0.093750,24,0.375000,"[(扶桑刺, 35), (法奶, 13), (攻辅射, 10), (蜀国, 2), (弟弟蜀..."
稷,4.736842,4.578947,5.212406,57,8,0.140351,19,0.333333,"[(扶桑刺, 34), (魔坦刺, 9), (魔种, 9), (蜀国, 3), (弟妹法, ..."
魏,4.240741,4.888889,5.270440,54,3,0.055556,17,0.314815,"[(扶桑刺, 19), (七射, 11), (稷下战, 10), (攻辅射, 9), (蜀国..."
吴,3.500000,5.428571,5.956044,14,2,0.142857,3,0.214286,"[(稷下战, 4), (扶桑刺, 3), (蜀国, 2), (九战, 2), (轻战守约, ..."
尧,2.833333,5.750000,4.568182,12,0,0.000000,3,0.250000,"[(七射, 7), (扶桑刺, 2), (攻辅射, 1), (稷下战, 1), (坦法, 1)]"
神,2.000000,6.000000,NaN,1,0,0.000000,0,0.000000,"[(魔坦刺, 1)]"


,平均分数,平均排名,排名方差,出场次数,登顶次数,登顶率,前三次数,前三率,阵容选用
俱乐部,,,,,,,,,
DYG,6.200000,3.800000,5.600000,15,4,0.266667,7,0.466667,"[(稷下战, 4), (扶桑刺, 3), (攻辅射, 3), (法奶, 1), (七射, 1..."
KS.YTG,6.100000,3.800000,5.511111,10,2,0.200000,5,0.500000,"[(扶桑刺, 6), (稷下战, 3), (攻辅射, 1)]"
CW,6.066667,3.933333,6.923810,15,4,0.266667,7,0.466667,"[(扶桑刺, 5), (弟弟射, 3), (魔种, 2), (稷下战, 2), (蜀国, 1..."
XQ,6.000000,3.900000,6.766667,10,1,0.100000,5,0.500000,"[(扶桑刺, 5), (弟弟射, 2), (攻辅射, 1), (稷下战, 1), (蜀国, 1)]"
KSSC,6.000000,3.933333,6.495238,15,3,0.200000,8,0.533333,"[(攻辅射, 7), (扶桑刺, 4), (魔种, 1), (九战, 1), (蜀国, 1)..."
成都AG超玩会,5.733333,3.933333,4.352381,15,2,0.133333,6,0.400000,"[(扶桑刺, 5), (弟弟射, 3), (蜀国, 2), (七射, 1), (扶桑法, 1..."
MQ,5.600000,4.300000,8.455556,10,3,0.300000,4,0.400000,"[(扶桑刺, 4), (魔种, 2), (稷下战, 2), (七射, 1), (弟弟射, 1)]"
重庆QGScholar,5.600000,4.400000,9.822222,10,3,0.300000,5,0.500000,"[(扶桑刺, 6), (弟弟射, 2), (稷下战, 2)]"
NMG,5.533333,4.066667,5.066667,15,2,0.133333,8,0.533333,"[(扶桑刺, 5), (弟弟射, 3), (稷下战, 2), (蜀国, 1), (攻辅射, ..."


,平均分数,平均排名,排名方差,出场次数,登顶次数,登顶率,前三次数,前三率,阵容选用
选手,,,,,,,,,
童小话,11.0,1.333333,0.333333,3,2,0.666667,3,1.000000,"[(扶桑刺, 3)]"
奕将,10.0,1.666667,0.333333,3,1,0.333333,3,1.000000,"[(弟弟射, 2), (稷下战, 1)]"
ray,8.5,2.500000,3.500000,6,2,0.333333,5,0.833333,"[(攻辅射, 5), (扶桑刺, 1)]"
简若,8.4,2.800000,7.200000,5,3,0.600000,3,0.600000,"[(弟弟射, 3), (扶桑刺, 1), (稷下战, 1)]"
噢吼,8.0,2.666667,2.333333,3,1,0.333333,2,0.666667,"[(扶桑刺, 1), (弟弟蜀, 1), (九战, 1)]"
...,...,...,...,...,...,...,...,...,...
花花,1.0,7.000000,NaN,1,0,0.000000,0,0.000000,"[(七射, 1)]"
莫然,1.0,7.000000,NaN,1,0,0.000000,0,0.000000,"[(九战, 1)]"
风流,1.0,7.000000,NaN,1,0,0.000000,0,0.000000,"[(扶桑刺, 1)]"


In [40]:
%%time

def report_mvp_comp(comp_stats):
    MIN_OCCURENCE = 5
    metric_weights = {
        '平均分数': [15, False],
        '平均排名': [5, True],
        '排名方差': [3, True],
        '出场次数': [5, False],
        '登顶率': [5, False],
        '前三率': [3, False]
    }
    comp_stats = comp_stats[comp_stats['出场次数'] >= MIN_OCCURENCE]
    raw_ranks = []
    weights = []
    for metric, value in metric_weights.items():
        weight, ascending = value
        metric_rank = comp_stats[metric].rank(ascending=ascending, method='min').astype(int)
        raw_ranks.append(metric_rank)
        weights.append(weight)
    comp_rank = sum([r * w for r, w in zip(raw_ranks, weights)]).sort_values()
    # print(comp_rank)
    mvp_comp = comp_rank.index[0]
    # print('本期MVP阵容:', mvp_comp)
    mvp_comp_metric_ranks = pd.Series({r.name: r[mvp_comp] for r in raw_ranks})
    mvp_comp_metric_values = comp_stats.loc[mvp_comp][mvp_comp_metric_ranks.index].T
    res = pd.concat([mvp_comp_metric_values, mvp_comp_metric_ranks], axis=1)
    res.columns = [mvp_comp, '全阵容对比排名']
    return res
comp_stats = get_comp_statistics(df)
display(report_mvp_comp(comp_stats))

def report_mvp_player(player_stats):
    MIN_OCCURENCE = 3
    metric_weights = {
        '平均分数': [15, False],
        '平均排名': [10, True],
        '排名方差': [5, True],
        '登顶率': [10, False],
        '前三率': [5, False],
        '出场次数': [0, False],
        '阵容选用': [0, False]
    }
    player_stats = player_stats[player_stats['出场次数'] >= MIN_OCCURENCE]
    raw_ranks = []
    weights = []
    for metric, value in metric_weights.items():
        weight, ascending = value
        metric_rank = player_stats[metric].rank(ascending=ascending, method='min').astype(int)
        raw_ranks.append(metric_rank)
        weights.append(weight)
    player_rank = sum([r * w for r, w in zip(raw_ranks, weights)]).sort_values()
    mvp_player = player_rank.index[0]
    # print('本期MVP选手:', mvp_player)
    mvp_player_metric_ranks = pd.Series({r.name: r[mvp_player] for r in raw_ranks})
    mvp_player_metric_values = player_stats.loc[mvp_player][mvp_player_metric_ranks.index].T
    res = pd.concat([mvp_player_metric_values, mvp_player_metric_ranks], axis=1)
    res.columns = [mvp_player, '全选手对比排名']
    res['全选手对比排名'] = (res['全选手对比排名'] / weights * weights).fillna('/')
    return res

player_stats = get_player_statistics(df)
display(report_mvp_player(player_stats))

def report_mvp_team(team_stats):
    metric_weights = {
        '平均分数': [15, False],
        '平均排名': [10, True],
        '排名方差': [5, True],
        '登顶率': [10, False],
        '前三率': [5, False],
        '出场次数': [0, False],
        '阵容选用': [0, False]
    }
    raw_ranks = []
    weights = []
    for metric, value in metric_weights.items():
        weight, ascending = value
        metric_rank = team_stats[metric].rank(ascending=ascending, method='min').astype(int)
        raw_ranks.append(metric_rank)
        weights.append(weight)
    team_rank = sum([r * w for r, w in zip(raw_ranks, weights)]).sort_values()
    mvp_team = team_rank.index[0]
    # print('本期MVP选手:', mvp_team)
    mvp_team_metric_ranks = pd.Series({r.name: r[mvp_team] for r in raw_ranks})
    mvp_team_metric_values = team_stats.loc[mvp_team][mvp_team_metric_ranks.index].T
    res = pd.concat([mvp_team_metric_values, mvp_team_metric_ranks], axis=1)
    res.columns = [mvp_team, '全队伍对比排名']
    res['全队伍对比排名'] = (res['全队伍对比排名'] / weights * weights).fillna('/')
    return res

team_stats = get_team_statistics(df)
display(report_mvp_team(team_stats))

,弟弟射,全阵容对比排名
平均分数,6.77778,1
平均排名,3.44444,1
排名方差,5.2254,5
出场次数,36,3
登顶率,0.25,1
前三率,0.583333,1


,童小话,全选手对比排名
平均分数,11,1
平均排名,1.33333,1
排名方差,0.333333,1
登顶率,0.666667,1
前三率,1,1
出场次数,3,/
阵容选用,"[(扶桑刺, 3)]",/


,DYG,全队伍对比排名
平均分数,6.2,1
平均排名,3.8,1
排名方差,5.6,13
登顶率,0.266667,3
前三率,0.466667,7
出场次数,15,/
阵容选用,"[(稷下战, 4), (扶桑刺, 3), (攻辅射, 3), (法奶, 1), (七射, 1...",/


Wall time: 164 ms


In [41]:
def report_interesting_data(df):
    b_df = df.copy()
    b_df['选手'] = b_df['选手'].astype(str)
    b_df['俱乐部'] = b_df['俱乐部'].astype(str)
    b_df['选手'] = b_df[['俱乐部', '选手']].agg('.'.join, axis=1)
    b_df['局数'] = b_df.apply(lambda x: f'第{x["周数"]}周{x.loc["分组"]}第{x.loc["局数"]}局', axis=1)
    # 最长游戏回合
    longest_game = b_df.loc[b_df['回合数'] == b_df['回合数'].max()][['选手', '详细阵容', '回合数', '局数', '排名']]
    print(f'最长游戏:\n{longest_game}\n')
    # 最短游戏回合
    first_twos = b_df.loc[b_df['排名'] <= 2]
    shortest_game = first_twos.loc[first_twos['回合数'] == first_twos['回合数'].min()][['选手', '详细阵容', '回合数', '局数', '排名']]
    print(f'最短游戏:\n{shortest_game}\n')
    highest_price = b_df.iloc[b_df['质量'].idxmax()][['选手', '详细阵容', '质量', '局数', '排名']]
    print(f'最高质量:\n{highest_price}\n')
    lowest_price = b_df.iloc[b_df['质量'].idxmin()][['选手', '详细阵容', '质量', '局数', '排名']]
    print(f'最低质量:\n{lowest_price}\n')
    highest_damage = b_df.iloc[b_df['输出'].idxmax()][['选手', '详细阵容', '输出', '局数', '排名']]
    print(f'最高输出:\n{highest_damage}\n')
    lowest_damage = b_df.iloc[b_df['输出'].idxmin()][['选手', '详细阵容', '输出', '局数', '排名']]
    print(f'最低输出:\n{lowest_damage}\n')

report_interesting_data(df)

最长游戏:
              选手    详细阵容  回合数         局数  排名
120  武汉eStar.小玄策  扶桑群雄法刺   36  第4周低分组第1局   1
121     XROCK.痘痘     弟弟射   36  第4周低分组第1局   2
144       RW侠.种子   稷下攻辅射   36  第4周低分组第4局   1
145   杭州LGD大鹅.六月     稷下战   36  第4周低分组第4局   2
296    XROCK.水长东    扶桑法刺   36  第5周中分组第3局   1
297     WB.TS.刺儿   封神攻辅射   36  第5周中分组第3局   2

最短游戏:
              选手   详细阵容  回合数         局数  排名
128  武汉eStar.曹老大     蜀国   26  第4周低分组第2局   1
129     WB.TS.刺儿    弟弟射   26  第4周低分组第2局   2
240   上海EDG.M.阿九    稷下战   26  第5周低分组第1局   1
241      ACT.胡老大    弟弟射   26  第5周低分组第1局   2
256     广州TTG.七璇    魔坦刺   26  第5周低分组第3局   1
257     KSSC.ray  封神攻辅射   26  第5周低分组第3局   2

最高质量:
选手          CW.简若
详细阵容         扶桑法刺
质量            177
局数      第3周中分组第1局
排名              1
Name: 40, dtype: object

最低质量:
选手        MQ.暴躁黑猪
详细阵容      魔种(无天赋)
质量             33
局数      第4周高分组第5局
排名              8
Name: 239, dtype: object

最高输出:
选手       广州TTG.七璇
详细阵容          魔坦刺
输出            261
局数      第5周低分组第3局
排名              1
Name: 256, dtype: object

In [42]:
def plot_ban_to_comp(ban_stats, filepath=None):
    fig, axs = plt.subplots(1, len(ban_stats), figsize=(30, 4))
    for ax, ban in zip(axs, ban_stats.itertuples()):
        labels, ys = [m[0] for m in ban[-1]], [m[1] for m in ban[-1]]
        colors = [comp_color[re.sub(r'\([^)]*\)', '', c).strip()] for c in labels]
        explode = [0.015] * len(ys)
        def value(val):
            return f'{val:.2f}%'
        ax.pie(ys, labels=labels, colors=colors, explode=explode, autopct=value, shadow=False, pctdistance=0.6)
        ax.set_title(f'{ban[0]} (均分{ban[1]:.2f},出场{ban[4]})', fontsize=15, x=0.5, y=1.01)

    fig.tight_layout()
    if filepath:
        fig.savefig(filepath, dpi=250)
    else:
        plt.show(fig)
    plt.close(fig)

def plot_comp_data(comp_stats, filepath=None):
    fig, ax = plt.subplots(figsize=(6, 4))
    y = comp_stats['平均排名']
    x = comp_stats['出场次数']
    names = comp_stats.index
    ax.scatter(x, y)
    ax.set_title('阵容数据')
    ax.set_ylabel('平均排名')
    ax.set_xlabel('出场次数')
    ax.set_xlim(left=0, right=max(x) + 1)
    ax.set_ylim(bottom=1, top=8)
    ax.hlines(y=4.5, xmin=0, xmax=max(x) + 1, colors='orange', linestyles='--', lw=2, label='理论均值')
    ax.legend(loc="lower right")
    for i, txt in enumerate(names):
        ax.annotate(txt, (x[i] + 0.25, y[i]))
    fig.gca().invert_yaxis()
    if filepath:
        fig.savefig(filepath, dpi=250)
    else:
        plt.show(fig)
    plt.close(fig)

comp_stats = get_comp_statistics(df)
plot_comp_data(comp_stats, "data/comp_data.png")
# plot_comp_data(comp_stats, None)
ban_stats = get_ban_statistics(df)
plot_ban_to_comp(ban_stats, "data/ban_to_comp.png")
# plot_ban_to_comp(ban_stats, None)

In [7]:
weibo_csv_path = 'weibo_crawler/weibo/王者模拟战职业大师赛/5464294919.csv'
weibo_df = pd.read_csv(weibo_csv_path)
weibo_df = weibo_df.loc[weibo_df['正文'].str.contains('决赛圈')]
display(weibo_df.head(1))
print(weibo_df.shape)



,id,bid,正文,头条文章url,原始图片url,视频url,位置,日期,工具,点赞数,评论数,转发数,话题,@用户
3,4558135123513004,JoomkeJTm,王者荣耀超话 KPL超话 #王者模拟战# #王者模拟战职业大师赛#·秋季赛·常规赛第四周...,NaN,https://wx4.sinaimg.cn/large/005XNCJ1gy1gjj3p5...,NaN,NaN,2020-10-09,搜狗高速浏览器,0,0,0,"王者模拟战,王者模拟战职业大师赛",NaN


(25, 14)
